In [183]:
import ndtest
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt

from scipy import stats
from pykrx import stock
from joblib import Parallel, delayed
from copula import *


In [138]:
tick_list = stock.get_market_ticker_list(date='2022-11-20', market='KOSPI')

In [139]:
tick_list = [ str(x)+'.KS' for x in tick_list ]

In [140]:
kospi_df = yf.download(tick_list)['Adj Close']

[*********************100%***********************]  943 of 943 completed

9 Failed downloads:
- 009275.KS: No data found, symbol may be delisted
- 101060.KS: No data found, symbol may be delisted
- 153360.KS: No data found, symbol may be delisted
- 172580.KS: No data found, symbol may be delisted
- 015350.KS: No data found, symbol may be delisted
- 007630.KS: No data found, symbol may be delisted
- 016385.KS: No data found, symbol may be delisted
- 138250.KS: No data found, symbol may be delisted
- 002270.KS: No data found, symbol may be delisted


In [141]:
kospi_df = kospi_df.sort_index(ascending=True)
kospi_df.index = pd.to_datetime(kospi_df.index)

In [142]:
kospi_df.loc['2012-01-02':]

,000020.KS,000040.KS,000050.KS,000060.KS,000070.KS,000075.KS,000080.KS,000087.KS,000100.KS,000105.KS,...,383220.KS,383800.KS,38380K.KS,395400.KS,396690.KS,400760.KS,402340.KS,404990.KS,900140.KS,950210.KS
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,3571.479004,4145.998047,6510.176758,7365.927734,63353.609375,22871.332031,18145.447266,11348.726562,17600.332031,18183.861328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6727.613770,NaN
2012-01-03,3610.822754,4739.997559,6538.146973,7623.251465,66666.742188,23779.457031,18593.908203,11348.726562,17531.312500,18183.861328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6687.805176,NaN
2012-01-04,3628.308105,5447.997070,6489.198730,7591.085449,66989.976562,23745.826172,18800.892578,11381.716797,17738.375000,18183.861328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6695.767090,NaN
2012-01-05,3733.223633,5213.997559,6524.162109,7462.424805,67394.015625,23510.382812,19283.849609,11315.738281,18083.482422,18629.546875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6608.188965,NaN
2012-01-06,3706.993896,5333.997559,6524.162109,7333.761719,67636.437500,23544.017578,19145.859375,11282.747070,17669.353516,18540.408203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6393.224121,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-14,9270.000000,593.000000,11050.000000,34250.000000,68600.000000,56500.000000,27100.000000,16500.000000,58000.000000,56500.000000,...,157000.0,9120.0,11550.0,4720.0,3450.0,3170.0,40500.0,3800.0,2975.000000,8340.0
2022-11-15,9550.000000,588.000000,10950.000000,34700.000000,69200.000000,56200.000000,26850.000000,16650.000000,59400.000000,56800.000000,...,158500.0,9310.0,11800.0,4660.0,3425.0,3190.0,40050.0,3810.0,3030.000000,8640.0
2022-11-16,9390.000000,583.000000,11000.000000,34650.000000,68600.000000,57000.000000,26950.000000,16600.000000,59100.000000,56900.000000,...,156500.0,9290.0,11600.0,4670.0,3415.0,3185.0,40450.0,3830.0,3020.000000,8510.0


In [143]:
kospi_df.loc['2012-01-02':].iloc[0].dropna().index

Index(['000020.KS', '000040.KS', '000050.KS', '000060.KS', '000070.KS',
       '000075.KS', '000080.KS', '000087.KS', '000100.KS', '000105.KS',
       ...
       '138490.KS', '138930.KS', '139130.KS', '139480.KS', '140910.KS',
       '145990.KS', '145995.KS', '161000.KS', '175330.KS', '900140.KS'],
      dtype='object', length=762)

In [144]:
tmp = kospi_df.loc['2012-01-02':]

In [145]:
tmp = tmp[kospi_df.loc['2012-01-02':].iloc[0].dropna().index]

In [146]:
tmp

,000020.KS,000040.KS,000050.KS,000060.KS,000070.KS,000075.KS,000080.KS,000087.KS,000100.KS,000105.KS,...,138490.KS,138930.KS,139130.KS,139480.KS,140910.KS,145990.KS,145995.KS,161000.KS,175330.KS,900140.KS
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,3571.479004,4145.998047,6510.176758,7365.927734,63353.609375,22871.332031,18145.447266,11348.726562,17600.332031,18183.861328,...,4421.835449,7145.692383,8114.371582,245899.796875,2807.818359,40549.800781,26004.554688,2760.902100,3179.941162,6727.613770
2012-01-03,3610.822754,4739.997559,6538.146973,7623.251465,66666.742188,23779.457031,18593.908203,11348.726562,17531.312500,18183.861328,...,4404.597168,7548.266602,8179.546387,256808.468750,2812.513672,40798.824219,25352.626953,2855.354004,3173.330322,6687.805176
2012-01-04,3628.308105,5447.997070,6489.198730,7591.085449,66989.976562,23745.826172,18800.892578,11381.716797,17738.375000,18183.861328,...,4352.878906,7313.433105,8146.959473,254535.843750,2770.255371,41213.871094,25895.902344,2844.455811,3133.664062,6695.767090
2012-01-05,3733.223633,5213.997559,6524.162109,7462.424805,67394.015625,23510.382812,19283.849609,11315.738281,18083.482422,18629.546875,...,4395.977051,7447.623535,8277.310547,251354.156250,2807.818359,40923.335938,25461.283203,2800.862305,3120.441650,6608.188965
2012-01-06,3706.993896,5333.997559,6524.162109,7333.761719,67636.437500,23544.017578,19145.859375,11282.747070,17669.353516,18540.408203,...,4568.368164,7246.335938,8440.250000,249990.578125,2765.560303,42334.492188,25280.191406,2684.614258,3074.164062,6393.224121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-14,9270.000000,593.000000,11050.000000,34250.000000,68600.000000,56500.000000,27100.000000,16500.000000,58000.000000,56500.000000,...,10700.000000,6760.000000,6940.000000,93000.000000,4090.000000,38900.000000,31050.000000,10500.000000,7430.000000,2975.000000
2022-11-15,9550.000000,588.000000,10950.000000,34700.000000,69200.000000,56200.000000,26850.000000,16650.000000,59400.000000,56800.000000,...,10750.000000,6820.000000,7050.000000,93600.000000,4135.000000,39100.000000,31150.000000,9880.000000,7500.000000,3030.000000
2022-11-16,9390.000000,583.000000,11000.000000,34650.000000,68600.000000,57000.000000,26950.000000,16600.000000,59100.000000,56900.000000,...,10900.000000,6900.000000,7290.000000,95700.000000,4150.000000,39050.000000,31600.000000,9610.000000,7720.000000,3020.000000


In [147]:
prices = tmp.copy()
prices.head()

,000020.KS,000040.KS,000050.KS,000060.KS,000070.KS,000075.KS,000080.KS,000087.KS,000100.KS,000105.KS,...,138490.KS,138930.KS,139130.KS,139480.KS,140910.KS,145990.KS,145995.KS,161000.KS,175330.KS,900140.KS
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,3571.479004,4145.998047,6510.176758,7365.927734,63353.609375,22871.332031,18145.447266,11348.726562,17600.332031,18183.861328,...,4421.835449,7145.692383,8114.371582,245899.796875,2807.818359,40549.800781,26004.554688,2760.902100,3179.941162,6727.613770
2012-01-03,3610.822754,4739.997559,6538.146973,7623.251465,66666.742188,23779.457031,18593.908203,11348.726562,17531.312500,18183.861328,...,4404.597168,7548.266602,8179.546387,256808.468750,2812.513672,40798.824219,25352.626953,2855.354004,3173.330322,6687.805176
2012-01-04,3628.308105,5447.997070,6489.198730,7591.085449,66989.976562,23745.826172,18800.892578,11381.716797,17738.375000,18183.861328,...,4352.878906,7313.433105,8146.959473,254535.843750,2770.255371,41213.871094,25895.902344,2844.455811,3133.664062,6695.767090
2012-01-05,3733.223633,5213.997559,6524.162109,7462.424805,67394.015625,23510.382812,19283.849609,11315.738281,18083.482422,18629.546875,...,4395.977051,7447.623535,8277.310547,251354.156250,2807.818359,40923.335938,25461.283203,2800.862305,3120.441650,6608.188965
2012-01-06,3706.993896,5333.997559,6524.162109,7333.761719,67636.437500,23544.017578,19145.859375,11282.747070,17669.353516,18540.408203,...,4568.368164,7246.335938,8440.250000,249990.578125,2765.560303,42334.492188,25280.191406,2684.614258,3074.164062,6393.224121


In [148]:
#returns = np.log(prices).diff().dropna()
returns = prices.pct_change().dropna()

In [149]:
returns.head()

,000020.KS,000040.KS,000050.KS,000060.KS,000070.KS,000075.KS,000080.KS,000087.KS,000100.KS,000105.KS,...,138490.KS,138930.KS,139130.KS,139480.KS,140910.KS,145990.KS,145995.KS,161000.KS,175330.KS,900140.KS
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,0.011016,0.143271,0.004296,0.034934,0.052296,0.039706,0.024715,0.000000,-0.003921,0.000000,...,-0.003898,0.056338,0.008032,0.044362,0.001672,0.006141,-0.025070,0.034211,-0.002079,-0.005917
2012-01-04,0.004842,0.149367,-0.007487,-0.004219,0.004849,-0.001414,0.011132,0.002907,0.011811,0.000000,...,-0.011742,-0.031111,-0.003984,-0.008849,-0.015025,0.010173,0.021429,-0.003817,-0.012500,0.001191
2012-01-05,0.028916,-0.042951,0.005388,-0.016949,0.006031,-0.009915,0.025688,-0.005797,0.019455,0.024510,...,0.009901,0.018348,0.016000,-0.012500,0.013559,-0.007049,-0.016783,-0.015326,-0.004219,-0.013080
2012-01-06,-0.007026,0.023015,0.000000,-0.017241,0.003597,0.001431,-0.007156,-0.002916,-0.022901,-0.004785,...,0.039216,-0.027027,0.019685,-0.005425,-0.015050,0.034483,-0.007112,-0.041504,-0.014830,-0.032530
2012-01-09,0.028302,-0.127109,-0.003216,-0.008772,-0.002389,0.000000,-0.001802,-0.008772,-0.023437,-0.019231,...,0.028302,0.004630,0.027027,-0.010909,-0.005093,-0.007843,0.005731,0.002706,0.000000,0.006226


In [150]:
prices = prices[prices.columns[0:50]]
returns = returns[returns.columns[0:50]]


trade_ratio = 0.3
start_trade = int(len(prices) * (1 - trade_ratio))

price_form = prices.iloc[:start_trade]
price_trade = prices.iloc[start_trade:]
returns_form = returns.iloc[:start_trade]
returns_trade = returns.iloc[start_trade:]

In [151]:
results = pd.DataFrame(columns=['tau'])

In [152]:
from tqdm import tqdm

for s1 in tqdm(returns_form.columns):
    for s2 in returns_form.columns:
        if (s1!=s2) and (f'{s2}-{s1}' not in results.index):
            results.loc[f'{s1}-{s2}'] = stats.kendalltau(returns_form[s1], returns_form[s2])[0]

100%|██████████| 50/50 [00:00<00:00, 53.91it/s]


In [158]:
results

,tau
000020.KS-000040.KS,0.115482
000020.KS-000050.KS,0.061974
000020.KS-000060.KS,0.059414
000020.KS-000070.KS,0.128877
000020.KS-000075.KS,0.093137
...,...
000720.KS-000760.KS,0.057645
000720.KS-000810.KS,0.084142
000725.KS-000760.KS,0.043308
000725.KS-000810.KS,0.040015


In [194]:
def parse_pair(pair):
    s1 = pair[:pair.find('-')]
    s2 = pair[pair.find('-')+1:]
    return s1,s2

In [207]:
selected_stocks = []
selected_pairs = []

for pair in results.sort_values(by='tau', ascending=False).index:
    s1,s2 = parse_pair(pair)
    if (s1 not in selected_stocks) and (s2 not in selected_stocks):
        selected_stocks.append(s1)
        selected_stocks.append(s2)
        selected_pairs.append(pair)
    
    if len(selected_pairs) == 100:
        break

In [208]:
len(selected_pairs)

25

In [209]:
marginals_df = pd.DataFrame(index=selected_stocks, columns=['Distribution', 'AIC', 'BIC', 'KS_pvalue'])

for stock in selected_stocks:
    data = returns_form[stock]
    dists = ['Normal', "Student's t", 'Logistic', 'Extreme']
    best_aic = np.inf
    for dist,name in zip([stats.norm, stats.t, stats.genlogistic, stats.genextreme], dists):
        params = dist.fit(data)
        dist_fit = dist(*params)
        log_like = np.log(dist_fit.pdf(data)).sum()
        aic = 2*len(params) - 2 * log_like
        if aic<best_aic:
            best_dist = name
            best_aic = aic
            best_bic = len(params) * np.log(len(data)) - 2 * log_like
            ks_pval = stats.kstest(data, dist_fit.cdf, N=100)[1]
    marginals_df.loc[stock] = [best_dist, best_aic, best_bic, ks_pval]

/var/folders/0k/zyr795gx3hl6x1nytrrntcnc0000gn/T/ipykernel_61631/271941337.py:10: RuntimeWarning: divide by zero encountered in log
  log_like = np.log(dist_fit.pdf(data)).sum()
/opt/homebrew/Caskroom/miniforge/base/envs/quant/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:2492: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)


In [210]:
marginals_df.head(5)

,Distribution,AIC,BIC,KS_pvalue
000210.KS,Student's t,-9033.047383,-9016.439892,0.112681
000215.KS,Student's t,-9633.879637,-9617.272145,0.003731
000220.KS,Student's t,-8306.076315,-8289.468824,0.002075
000225.KS,Student's t,-9151.71435,-9135.106859,0.008733
000150.KS,Student's t,-9317.447936,-9300.840445,0.002811


In [211]:
marginals_df['Distribution'].unique()

array(["Student's t", 'Logistic'], dtype=object)

In [212]:
marginals_df[marginals_df['KS_pvalue'] > 0.05].index.to_list()

['000210.KS', '000720.KS', '000270.KS', '000430.KS', '000670.KS', '000660.KS']

In [217]:
selected_pairs[0][:9]

'000210.KS'

In [218]:
marg_selected_pairs = []

for x in selected_pairs:
    if x[:9] in marginals_df[marginals_df['KS_pvalue'] > 0.05].index.to_list():
        marg_selected_pairs.append(x)

In [220]:
selected_pairs = marg_selected_pairs

In [227]:
from statsmodels.distributions.copula.api import GaussianCopula, GumbelCopula, ClaytonCopula, FrankCopula


In [ ]:
from scipy.copu

In [228]:
copulas_df = pd.DataFrame(index=selected_pairs, columns=['copula', 'parameter', 'aic', 'bic', 'KS_pvalue'])

for pair in selected_pairs:
    s1,s2 = parse_pair(pair)
    # fit marginals
    params_s1 = stats.t.fit(returns_form[s1])
    dist_s1 = stats.t(*params_s1)
    params_s2 = stats.t.fit(returns_form[s2])
    dist_s2 = stats.t(*params_s2)
    # apply probability integral transform
    u = dist_s1.cdf(returns_form[s1])
    v = dist_s2.cdf(returns_form[s2])
    
    best_aic = np.inf
    
    for copula in [GaussianCopula(), ClaytonCopula(), GumbelCopula(), FrankCopula(), JoeCopula()]:
        copula.fit(u,v)
        L = copula.log_likelihood(u,v)
        aic = 2 * copula.num_params - 2 * L
        if aic < best_aic:
            best_aic = aic
            best_bic = copula.num_params * np.log(len(u)) - 2 * L
            best_copula = copula.name
            
            # calculate KS-pvalue
            smp = copula.sample(size=len(u)) # generate sample from fit copula
            s_u = smp[:,0]
            s_v = smp[:,1]
            ks_pval = ndtest.ks2d2s(u,v,s_u,s_v)
            
            if isinstance(copula, ArchimedeanCopula):
                best_param = copula.alpha
            else:
                best_param = copula.rho
            
    copulas_df.loc[pair] = [best_copula, best_param, best_aic, best_bic, ks_pval]

AttributeError: 'GaussianCopula' object has no attribute 'fit'